In [ ]:
from kedro_datasets.pandas import ParquetDataset
import pandas as pd

#kobe_raw_dev = catalog.load(name='kobe_raw_dev@parquet')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
dataset = ParquetDataset(filepath='../data/01_raw/dataset_kobe_dev.parquet').load()

In [ ]:
dataset = dataset.drop(columns=['action_type','combined_shot_type','game_event_id','game_id','loc_x','loc_y','season','seconds_remaining','shot_type','shot_zone_area','shot_zone_basic','shot_zone_range','team_id','team_name','game_date','matchup','opponent','shot_id'])

In [ ]:
dataset.info()

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset.info()

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset['shot_made_flag'] = dataset['shot_made_flag'].apply(lambda r: 1 if r == 1.0 else 0)

In [ ]:
dataset['shot_made_flag'].value_counts()

In [ ]:
dataset.head()


In [ ]:
from pycaret.classification import *
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
experiment = ClassificationExperiment()
experiment.setup(
    data=dataset, 
    target='shot_made_flag', 
    log_experiment='mlflow',
    experiment_name='kobe_pycarret',
    #log_profile=True,
    #log_data=True,
    #normalize=True,
    #normalize_method='robust',
    #polynomial_features=True,
    #feature_selection=True,
    #remove_multicollinearity=True,
    #pca=True,
)

In [ ]:
experiment.get_metrics()

In [ ]:
best = experiment.compare_models(n_select=15)

In [ ]:
#scikit-learn, scikit-optimize, tune-sklearn, optuna
model_1 = experiment.tune_model(
        experiment.create_model('dt'),
        optimize='F1',
        n_iter=10,
        search_library='scikit-optimize',
    )
experiment.plot_model(model_1, plot = 'auc')


In [ ]:
model_2 = experiment.tune_model(
    experiment.create_model('lr'),
    optimize='F1',
    n_iter=10,
    search_library='scikit-learn',
)
experiment.plot_model(model_2, plot = 'auc')

In [ ]:
experiment.evaluate_model(best)

In [ ]:
import pickle

with open('model.pkl', 'wb') as fout:
    pickle.dump(best, fout)

In [ ]:
#experiment.plot_model(best, plot = 'residuals_interactive')

In [ ]:
experiment.plot_model(best, plot = 'feature')

In [ ]:
final_best = experiment.finalize_model(best)

In [ ]:
final_best